In [1]:
# 누락된 문서ID 찾기 + 상세 정보 크롤링
# txt 파일의 문서ID 중 DB에 없는 것을 찾고, 웹에서 상세 정보 수집
'''
전체 흐름
doc_ids.txt에서 문서ID 목록 로드 → DB 비교 → 누락 ID 확인 → 웹 크롤링 → 상세 정보 수집 

1단계: 누락된 문서 찾기
작업/설명
txt 파일 읽기/doc_ids.txt에서 문서ID 목록 로드
DB 조회/approval_data_plus 테이블의 document_id 조회
차집합 계산/txt에는 있는데 DB에는 없는 문서ID 추출
txt 저장/누락된 ID만 missing_doc_ids.txt로 저장 (기존 기능 유지)

2단계: 웹에서 상세 정보 크롤링
작업/설명
브라우저 실행/Chrome 열고 로그인 페이지 이동
수동 작업 대기/로그인 → 메뉴 진입 → 필터 설정 → 조회 (사용자가 직접)
그리드 순회/페이지별로 목록 스캔
누락 문서만 수집/각 행의 문서ID가 누락 목록에 있으면 → 문서번호, 제목, 기안자, 기안일 추출
조기 종료/모든 누락 문서를 찾으면 더 이상 페이지 넘기지 않음

3단계: 결과 저장
출력 파일/내용
missing_doc_ids.txt/누락된 문서ID만 (기존 형식 유지)
missing_docs_detail.csv/문서ID, 제목, 기안자, 기안일

🔧 설정 부분 외에 ctrl+f -> #테이블명 수정하세요 검색해서 테이블명도 수정하세요
'''
import pymysql
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv
import re

# ═══════════════════════════════════════════════════════════
# 🔧 설정
# ═══════════════════════════════════════════════════════════

BASE_URL = "http://office.anyfive.com/home/"

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval'
}

# 기간 설정 (크롤링 시 필터용)
START_DATE = "2025-11-01"  # 여기를 수정하세요
END_DATE = "2025-12-31"    # 여기를 수정하세요

# 입력 파일 (크롤링한 문서ID 목록)
INPUT_FILE = "doc_ids_1231_plus.txt"

# 출력 파일
OUTPUT_TXT = "missing_doc_ids_12311_plus.txt"        # 누락된 ID만
OUTPUT_CSV = "missing_docs_detail_12311_plus.csv"    # 상세 정보 포함

# ═══════════════════════════════════════════════════════════
# 📦 유틸리티 함수
# ═══════════════════════════════════════════════════════════

def norm_space(s):
    """공백 정규화"""
    if not s: return ""
    s = s.replace("\xa0", " ")
    s = re.sub(r'[\t\r\n]+', ' ', s)
    s = re.sub(r'\s+', ' ', s)
    return s.strip()

def switch_into_frame_having(driver, selector_css, max_retries=3):
    """selector가 있는 프레임으로 전환"""
    for attempt in range(max_retries):
        driver.switch_to.default_content()
        
        try:
            WebDriverWait(driver, 5).until(
                EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
            )
        except:
            pass
        
        time.sleep(1)
        
        if driver.find_elements(By.CSS_SELECTOR, selector_css):
            print(f"  ✓ 그리드 발견 (최상위)")
            return True
        
        frames = driver.find_elements(By.CSS_SELECTOR, "iframe, frame")
        for idx, fr in enumerate(frames):
            try:
                driver.switch_to.default_content()
                driver.switch_to.frame(fr)
                if driver.find_elements(By.CSS_SELECTOR, selector_css):
                    print(f"  ✓ 그리드 발견 (프레임 {idx})")
                    return True
            except:
                continue
        
        if attempt < max_retries - 1:
            print(f"  ⚠️ 재시도 {attempt + 1}/{max_retries}")
            time.sleep(2)
    
    return False

def wait_grid_stable(driver, timeout=10):
    """그리드 안정화 대기"""
    try:
        WebDriverWait(driver, timeout).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "table[role='grid'] tbody tr"))
        )
        time.sleep(1)
        return True
    except:
        return False

# ═══════════════════════════════════════════════════════════
# 🕷️ 그리드 파싱 (누락된 문서만 필터링)
# ═══════════════════════════════════════════════════════════

def parse_grid_for_missing(driver, missing_ids):
    """현재 페이지에서 누락된 문서만 추출"""
    table = driver.find_element(By.CSS_SELECTOR, "table[role='grid']")
    
    # 헤더 분석
    th_all = table.find_elements(By.CSS_SELECTOR, "thead th")
    field_index = {}
    for i, th in enumerate(th_all):
        field = (th.get_attribute("data-field") or "").strip()
        if field:
            field_index[field] = i
    
    # 행 파싱
    rows = table.find_elements(By.CSS_SELECTOR, "tbody tr[role='row']")
    results = []
    
    for tr in rows:
        tds = tr.find_elements(By.CSS_SELECTOR, "td")
        if not tds:
            continue
        
        # 체크박스에서 문서ID 추출
        cb = tr.find_elements(By.CSS_SELECTOR, "td input[name='aprDoc_grid_ckbx']")
        doc_id = cb[0].get_attribute("value").strip() if cb else ""
        
        # 누락된 문서만 수집
        if doc_id not in missing_ids:
            continue
        
        def get_td_text(field, fallback_idx=None):
            idx = field_index.get(field)
            if idx is not None and idx < len(tds):
                return norm_space(tds[idx].text or tds[idx].get_attribute("textContent") or "")
            if fallback_idx is not None and fallback_idx < len(tds):
                return norm_space(tds[fallback_idx].text or tds[fallback_idx].get_attribute("textContent") or "")
            return ""
        
        item = {
            "문서ID": doc_id,
            "양식명": get_td_text("form_nm", 1),
            "문서번호": get_td_text("doc_sn", 2),
            "제목": get_td_text("appr_doc_tit", 3),
            "기안자": get_td_text("kor_nm", 4),
            "기안일": get_td_text("drft_dtm", 5),
        }
        
        results.append(item)
    
    return results

# ═══════════════════════════════════════════════════════════
# 🔄 페이징
# ═══════════════════════════════════════════════════════════

def has_next_page(driver):
    """다음 페이지 존재 여부"""
    try:
        next_btns = driver.find_elements(By.CSS_SELECTOR, 
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        return len(next_btns) > 0
    except:
        return False

def go_to_next_page(driver):
    """다음 페이지로 이동"""
    try:
        next_btn = driver.find_element(By.CSS_SELECTOR,
            "a.k-pager-nav[aria-label='다음 페이지로 이동']:not(.k-state-disabled)")
        
        driver.execute_script("arguments[0].click();", next_btn)
        time.sleep(2)
        
        WebDriverWait(driver, 15).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, ".k-loading-mask"))
        )
        
        wait_grid_stable(driver)
        return True
    except Exception as e:
        print(f"  ❌ 페이지 이동 실패: {e}")
        return False

# ═══════════════════════════════════════════════════════════
# 🚀 메인
# ═══════════════════════════════════════════════════════════

def main():
    print("=" * 60)
    print("🔍 누락된 문서ID 찾기 + 상세 정보 크롤링")
    print("=" * 60)
    
    # ─────────────────────────────────────────────────────────
    # 1단계: txt vs DB 비교
    # ─────────────────────────────────────────────────────────
    print(f"\n📄 {INPUT_FILE} 읽는 중...")
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        content = f.read().strip()
    
    txt_ids = set()
    for item in content.split(","):
        doc_id = item.strip().strip('"')
        if doc_id:
            txt_ids.add(doc_id)
    
    print(f"  ✓ txt 파일: {len(txt_ids)}건")
    
    print(f"\n🗄️ DB 조회 중...")
    conn = pymysql.connect(**DB_CONFIG)
    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT document_id FROM new_approval_data_plus") #테이블명 수정하세요
            rows = cursor.fetchall()
            db_ids = set(str(row[0]) for row in rows if row[0])
    finally:
        conn.close()
    
    print(f"  ✓ DB 테이블: {len(db_ids)}건")
    
    # 차집합 계산
    missing_ids = txt_ids - db_ids
    
    print(f"\n{'=' * 60}")
    print(f"📊 비교 결과")
    print(f"{'=' * 60}")
    print(f"  • txt 파일: {len(txt_ids)}건")
    print(f"  • DB 테이블: {len(db_ids)}건")
    print(f"  • 누락된 문서: {len(missing_ids)}건")
    
    if not missing_ids:
        print(f"\n✅ 모든 문서가 DB에 있습니다!")
        return
    
    # 누락된 ID만 저장 (기존 기능 유지)
    sorted_ids = sorted(missing_ids, key=lambda x: int(x) if x.isdigit() else 0, reverse=True)
    output_text = ",".join(f'"{doc_id}"' for doc_id in sorted_ids)
    with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
        f.write(output_text)
    print(f"\n📄 ID 목록 저장: {OUTPUT_TXT}")
    
    # ─────────────────────────────────────────────────────────
    # 2단계: 웹에서 상세 정보 크롤링
    # ─────────────────────────────────────────────────────────
    print(f"\n{'=' * 60}")
    print(f"🕷️ 누락된 문서 상세 정보 크롤링")
    print(f"{'=' * 60}")
    
    # 브라우저 시작
    opts = webdriver.ChromeOptions()
    opts.add_argument("--start-maximized")
    opts.add_argument("--disable-popup-blocking")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    driver = webdriver.Chrome(service=Service(), options=opts)
    
    try:
        driver.get(BASE_URL)
        time.sleep(1)
        
        print("\n" + "=" * 60)
        print("📋 수동 작업이 필요합니다!")
        print("=" * 60)
        print("다음 단계를 완료한 후 Enter를 눌러주세요:\n")
        print("  1️⃣  로그인")
        print("  2️⃣  전자결재 메뉴 클릭")
        print("  3️⃣  결재 문서관리 클릭")
        print(f"  4️⃣  상태 = 완료 선택")
        print(f"  5️⃣  시작일 = {START_DATE}")
        print(f"  6️⃣  종료일 = {END_DATE}")
        print("  7️⃣  조회 버튼 클릭 → 그리드 나타남 확인")
        print("\n💡 그리드가 화면에 보이면 Enter를 누르세요!")
        print("=" * 60)
        input("\n✋ 준비되면 Enter... ")
        
        time.sleep(1)
        
        if not switch_into_frame_having(driver, "table[role='grid']"):
            print("❌ 그리드를 찾을 수 없습니다")
            return
        
        wait_grid_stable(driver)
        
        # 페이지별 순회하며 누락된 문서 수집
        found_docs = []
        found_ids = set()
        current_page = 1
        
        while True:
            print(f"\n📄 페이지 {current_page} 스캔 중...")
            
            if not switch_into_frame_having(driver, "table[role='grid']"):
                print("⚠️ 그리드 재탐색 실패")
                break
            
            wait_grid_stable(driver)
            
            # 누락된 문서만 추출
            page_docs = parse_grid_for_missing(driver, missing_ids)
            
            if page_docs:
                found_docs.extend(page_docs)
                for doc in page_docs:
                    found_ids.add(doc["문서ID"])
                print(f"  ✓ 누락 문서 {len(page_docs)}건 발견 (누적: {len(found_docs)}건)")
            
            # 모두 찾았으면 조기 종료
            if found_ids == missing_ids:
                print(f"\n🎉 모든 누락 문서를 찾았습니다!")
                break
            
            # 다음 페이지
            if has_next_page(driver):
                if go_to_next_page(driver):
                    current_page += 1
                else:
                    break
            else:
                print(f"\n📄 마지막 페이지 도달")
                break
        
        # ─────────────────────────────────────────────────────
        # 3단계: 결과 출력 및 저장
        # ─────────────────────────────────────────────────────
        print(f"\n{'=' * 60}")
        print(f"📊 크롤링 결과")
        print(f"{'=' * 60}")
        print(f"  • 누락된 문서: {len(missing_ids)}건")
        print(f"  • 찾은 문서: {len(found_docs)}건")
        
        not_found = missing_ids - found_ids
        if not_found:
            print(f"  • 못 찾은 문서: {len(not_found)}건")
        
        if found_docs:
            # CSV 저장
            with open(OUTPUT_CSV, "w", encoding="utf-8-sig", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=["문서ID", "양식명", "문서번호", "제목", "기안자", "기안일"])
                writer.writeheader()
                writer.writerows(found_docs)
            
            print(f"\n✅ 상세 정보 저장: {OUTPUT_CSV}")
            
            # 화면 출력
            print(f"\n{'─' * 100}")
            print(f"{'문서ID':<12} {'제목':<40} {'기안자':<10} {'기안일':<12}")
            print(f"{'─' * 100}")
            for doc in found_docs[:20]:  # 처음 20개만 표시
                title = doc["제목"][:38] + ".." if len(doc["제목"]) > 40 else doc["제목"]
                print(f"{doc['문서ID']:<12} {title:<40} {doc['기안자']:<10} {doc['기안일']:<12}")
            
            if len(found_docs) > 20:
                print(f"  ... 외 {len(found_docs) - 20}건")
            print(f"{'─' * 100}")
        
        print(f"\n{'=' * 60}")
        
    except Exception as e:
        print(f"\n❌ 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        input("\n✋ 브라우저를 닫으려면 Enter를 누르세요...")
        driver.quit()

if __name__ == "__main__":
    main()

🔍 누락된 문서ID 찾기 + 상세 정보 크롤링

📄 doc_ids_1231.txt 읽는 중...
  ✓ txt 파일: 206건

🗄️ DB 조회 중...
  ✓ DB 테이블: 206건

📊 비교 결과
  • txt 파일: 206건
  • DB 테이블: 206건
  • 누락된 문서: 0건

✅ 모든 문서가 DB에 있습니다!


In [2]:
# 누락된 문서ID 찾기
# txt 파일의 문서ID 중 DB에 없는 것을 추출

import pymysql

# ═══════════════════════════════════════════════════════════
# 🔧 설정
# ═══════════════════════════════════════════════════════════

# DB 설정
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'any_approval'
}

# 입력 파일 (크롤링한 문서ID 목록)
INPUT_FILE = "doc_ids.txt"

# 출력 파일 (누락된 문서ID)
OUTPUT_FILE = "missing_doc_ids.txt"

# ═══════════════════════════════════════════════════════════
# 🚀 메인
# ═══════════════════════════════════════════════════════════

def main():
    print("=" * 60)
    print("🔍 누락된 문서ID 찾기")
    print("=" * 60)
    
    # 1. txt 파일에서 문서ID 읽기
    print(f"\n📄 {INPUT_FILE} 읽는 중...")
    with open(INPUT_FILE, "r", encoding="utf-8") as f:
        content = f.read().strip()
    
    # "26573755","26573742",... 형식 파싱
    txt_ids = set()
    for item in content.split(","):
        doc_id = item.strip().strip('"')
        if doc_id:
            txt_ids.add(doc_id)
    
    print(f"  ✓ txt 파일: {len(txt_ids)}건")
    
    # 2. DB에서 document_id 조회
    print(f"\n🗄️ DB 조회 중...")
    conn = pymysql.connect(**DB_CONFIG)
    try:
        with conn.cursor() as cursor:
            cursor.execute("SELECT document_id FROM approval_data_plus")
            rows = cursor.fetchall()
            db_ids = set(str(row[0]) for row in rows if row[0])
    finally:
        conn.close()
    
    print(f"  ✓ DB 테이블: {len(db_ids)}건")
    
    # 3. 차집합 계산 (txt에는 있는데 DB에는 없는 것)
    missing_ids = txt_ids - db_ids
    
    print(f"\n{'=' * 60}")
    print(f"📊 결과")
    print(f"{'=' * 60}")
    print(f"  • txt 파일: {len(txt_ids)}건")
    print(f"  • DB 테이블: {len(db_ids)}건")
    print(f"  • 누락된 문서: {len(missing_ids)}건")
    
    # 4. 결과 저장
    if missing_ids:
        # 정렬해서 저장
        sorted_ids = sorted(missing_ids, key=lambda x: int(x) if x.isdigit() else 0, reverse=True)
        output_text = ",".join(f'"{doc_id}"' for doc_id in sorted_ids)
        
        with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
            f.write(output_text)
        
        print(f"\n✅ 저장 완료: {OUTPUT_FILE}")
        
        # 미리보기 (처음 10개)
        print(f"\n📋 미리보기 (처음 10개):")
        for doc_id in sorted_ids[:10]:
            print(f"  - {doc_id}")
        if len(sorted_ids) > 10:
            print(f"  ... 외 {len(sorted_ids) - 10}건")
    else:
        print(f"\n✅ 모든 문서가 DB에 있습니다!")
    
    print(f"\n{'=' * 60}")

if __name__ == "__main__":
    main()

🔍 누락된 문서ID 찾기

📄 doc_ids.txt 읽는 중...
  ✓ txt 파일: 1785건

🗄️ DB 조회 중...
  ✓ DB 테이블: 23320건

📊 결과
  • txt 파일: 1785건
  • DB 테이블: 23320건
  • 누락된 문서: 1건

✅ 저장 완료: missing_doc_ids.txt

📋 미리보기 (처음 10개):
  - 26415181

